In [4]:
import os
import numpy as np
from sklearn.utils import shuffle
from pandas.io.parsers import read_csv

In [71]:
train_file = './data/training.csv'
testing_file = './data/testing.csv'

def loadTrainFile():
    df = read_csv(os.path.expanduser(train_file))
    
    # Es un solo campo separado por espacios hay que mostrarlo como array
    df['Image'] = df['Image'].apply(lambda im: np.fromstring(im, sep=' '))
    df = df.dropna() 
    
    #Escalar los pixeles entre 0 y 1
    X = np.vstack(df['Image'].values) / 255. 
    X = X.astype(np.float32)
    
    y = df[df.columns[:-1]].values
    y = (y - 48) / 48  # scale target coordinates to [-1, 1]
    X, y = shuffle(X, y, random_state=42)  # shuffle train data
    y = y.astype(np.float32)
    
    #escalar en 2 dimensiones, los pixeles son de 96 x 96
    X = X.reshape(-1, 96, 96, 1)
    return X, y

X, y = loadTrainFile()

In [72]:
# El tipo de red neuronal a implementar es LeNet-5  (http://yann.lecun.com/exdb/lenet/)
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten, Dropout
from keras.optimizers import SGD
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.callbacks import EarlyStopping, LearningRateScheduler
from keras.preprocessing.image import ImageDataGenerator

def buildModel():

    model = Sequential()

    model.add(Convolution2D(32,(3,3), input_shape=(96, 96, 1)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.1))
    

    model.add(Convolution2D(64, (2, 2)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Convolution2D(128, (2, 2)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.3))

    model.add(Flatten())
    model.add(Dense(1000))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1000))
    model.add(Activation('relu'))
    model.add(Dense(30))

    return model

In [73]:
from keras.preprocessing.image import ImageDataGenerator
class FlippedImageDataGenerator(ImageDataGenerator):
    flip_indices = [(0, 2), (1, 3), (4, 8), (5, 9),
                    (6, 10), (7, 11), (12, 16), (13, 17),
                    (14, 18), (15, 19), (22, 24), (23, 25)]

    def next(self):
        X_batch, y_batch = super(FlippedImageDataGenerator, self).next()
        batch_size = X_batch.shape[0]
        indices = np.random.choice(batch_size, batch_size / 2, replace=False)
        X_batch[indices] = X_batch[indices, :, :, ::-1]

        if y_batch is not None:
            y_batch[indices, ::2] = y_batch[indices, ::2] * -1

            for a, b in self.flip_indices:
                y_batch[indices, a], y_batch[indices, b] = (
                    y_batch[indices, b], y_batch[indices, a]
                )

        return X_batch, y_batch

In [74]:
from sklearn.model_selection import train_test_split
from keras.optimizers import SGD

start = 0.03
stop = 0.001
nb_epoch = 1000
learning_rate = np.linspace(start, stop, nb_epoch)

X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2, random_state=42)

net = buildModel()
sgd = SGD(lr=start, momentum=0.9, nesterov=True)
net.compile(loss='mse', optimizer=sgd)
change_lr = LearningRateScheduler(lambda epoch: float(learning_rate[epoch]))
early_stop = EarlyStopping(patience=100)

flipgen = FlippedImageDataGenerator()
hist = net.fit_generator(flipgen.flow(X_train, y_train),
                        steps_per_epoch=X_train.shape[0],
                        epochs=nb_epoch,
                        validation_data=(X_test, y_test),
                        callbacks=[change_lr, early_stop])



Epoch 1/1000
1712/1712 [==============================] - 718s - loss: 0.0055 - val_loss: 0.0044
Epoch 2/1000
1712/1712 [==============================] - 729s - loss: 0.0040 - val_loss: 0.0034
Epoch 3/1000
1712/1712 [==============================] - 737s - loss: 0.0032 - val_loss: 0.0026
Epoch 4/1000
1712/1712 [==============================] - 737s - loss: 0.0026 - val_loss: 0.0021
Epoch 5/1000
1712/1712 [==============================] - 738s - loss: 0.0022 - val_loss: 0.0019
Epoch 6/1000
1712/1712 [==============================] - 745s - loss: 0.0020 - val_loss: 0.0017
Epoch 7/1000
1712/1712 [==============================] - 745s - loss: 0.0019 - val_loss: 0.0016
Epoch 8/1000
1712/1712 [==============================] - 744s - loss: 0.0017 - val_loss: 0.0015
Epoch 9/1000
1712/1712 [==============================] - 748s - loss: 0.0016 - val_loss: 0.0015
Epoch 10/1000
1712/1712 [==============================] - 749s - loss: 0.0016 - val_loss: 0.0014
Epoch 11/1000
1712/1712 [====

KeyboardInterrupt: 